In [ ]:
import astro_amase
import pandas as pd

# Full Algorithm Examples

The following few cells will give some examples of different use cases for the various function. Note: the directory_path should lead to where the required Drobpox files are saved. The outputs will also be stored here.

## Key Parameter Explanations

### Required Parameters
- **`spectrum_path`**: Path to your spectrum file (two-column text file: frequency in MHz, intensity in K)
- **`directory_path`**: Directory where the required database files from Dropbox are saved. All output files will also be written to this directory
- **`observation_type`**: Either `'interferometric'` or `'single_dish'`
  - If interferometric: must provide `beam_major_axis` and `beam_minor_axis` (arcsec)
  - If single dish: must provide `dish_diameter` (meters)

### Source Parameter Settings
- **`temperature`**: Excitation temperature in Kelvin (initial guess if `temperature_is_exact=False`)
- **`temperature_is_exact`**: 
  - `True` = Use the provided temperature value as fixed (required if vlsr is fixed).
  - `False` = Optimize temperature automatically (initial guess can vary ~100K)
- **`vlsr`**: Source velocity (km/s)
  - `None` = Code will automatically determine the source velocity (default)
  - Numeric value = Use fixed source velocity
- **`continuum_temperature`**: Background continuum temperature (K), default is 2.7 (CMB)
- **`linewidth`**: Linewidth (FWHM) in km/s
  - `None` = Code will automatically determine the linewidth (default). Typically this is quite fast and reliable.
  - Numeric value = Use fixed linewidth

### Detection Settings
- **`sigma_threshold`**: Signal-to-noise cutoff for peak detection (e.g., 5 = detect lines ≥5σ). Default is 5 if not inputted. Note that it is not recommended to use too low of a sigma_threshold cutoff since during the fitting process, all molecules whose maximum intensity is < 2.5σ are removed from the assignment.
- **`rms_noise`**: 
  - `None` = Automatically determine noise level from spectrum (default)
  - Numeric value = Use fixed RMS noise value (same units as intensity). If a numeric value is entered, this noise level will be used for the peak finding in the entire spectrum. If `None` is inputted, the code will attempt to find and use the local noise level in each part of the spectrum.

### Molecular Assignment Settings
- **`valid_atoms`**: (Optional) List restricting molecular candidates to only those containing these elements (default `['C', 'O', 'H', 'N', 'S']`)
- **`known_molecules`**: (Optional) List of SMILES strings for molecules likely present in your source (e.g., `['C=O', 'CO', 'CCO']` for formaldehyde, methanol, and ethanol). These initialize the structural relevance scoring algorithm, which prioritizes chemically similar candidates during assignment. Duplicates can be included to increase weighting (e.g., `['C=O', 'C=O']`). **Note:** Without this parameter, the algorithm makes initial assignments based purely on spectroscopic matching; structural relevance scoring only activates after the first successful assignment. The list grows dynamically as additional molecules are assigned.
- **`force_include_molecules`**: (Optional) List of molecule names to force into the final fit
- **`force_ignore_molecules`**: (Optional) List of molecule names to exclude from consideration
- **`fitting_iterations`**: (Optional) Number of least-squares fitting iterations to perform following iterative line assignment. Each iteration after the first applies molecule filtering followed by refitting. Set to 0 to automatically iterate until convergence (when no molecules are removed). Otherwise, must be 2 or greater. Default: 0

### Output File Location Settings
- **`save_name`**: (Optional, str) The directory directory_path/save_name will be created to save output files. Default: 'no_name'
- **`overwrite_old_files`**: (Optional, bool) If directory_path/save_name already exists, and overwrite_old_files = True, the old output files will overwritten with the new results. If the path exists and overwrite_old_files = False, a temporary folder will be created to save the outputs. Default: False


##### For additional optional parameters, see the source code!

In [ ]:
'''
Example in which interferometric observation is used and neither the vlsr the temperature are known (so will be estimated by code).
'''


results = astro_amase.assign_observations(spectrum_path='PATH TO SPECTRUM',
                                          directory_path = 'DIRECTORY PATH',
                                          temperature=150,  # Initial guess
                                          temperature_is_exact=False, #will estimate temperature automatically
                                          vlsr = None, #code will automatically determine vlsr
                                          sigma_threshold = 5, #5 sigma lines and stronger
                                          rms_noise = None, #will automatically determine noise level (Recommended)
                                          observation_type = 'interferometric',
                                          beam_major_axis = 0.4, #arcsec
                                          beam_minor_axis = 0.4, #arcsec
                                          continuum_temperature = 13,#default to 2.7
                                          source_size = 1E20, #default to 1E20
                                          valid_atoms = ['C', 'O', 'H', 'N', 'S'],
                                          save_name = 'name_of_source') #will create the subfolder directory_path/save_name to save output files

assigner =results['assigner'] #Object used to assign the lines
for _ in range(5):
    print()
print('============================================================')
print('Summary Statistics!')
print('============================================================')
print(results['statistics'])
print('vlsr', results['vlsr'])
print('temperature', results['temperature'])

for line in assigner.lines[:10]:  # First 10 lines
    if line.assignment_status:
        print(f"{line.frequency:.4f} MHz: {line.assignment_status.value}")
        if line.assigned_molecule: 
            print(f"  → {line.assigned_molecule}")
            print(f"  Score: {line.best_candidate.global_score:.2f}")
            print('All Candidates:') #printing the molecular formula of all candidate transitions
            candidate_list = [i.formula for i in line.candidates]
            print(candidate_list)
            print('')

In [ ]:
'''
Example in which single dish observation is used and neither the vlsr the temperature are known (so will be estimated by code).
'''

results = astro_amase.assign_observations(spectrum_path='PATH TO SPECTRUM',
                                          directory_path = 'DIRECTORY PATH',
                                          temperature=170,  # Initial guess
                                          temperature_is_exact=False, #will estimate temperature automatically
                                          vlsr = None, #code will automatically determine vlsr
                                          sigma_threshold = 5, #5 sigma lines and stronger
                                          rms_noise = None, #will automatically determine noise level
                                          observation_type = 'single_dish',
                                          dish_diameter = 100, #meters
                                          continuum_temperature = 13,#default to 2.7
                                          source_size = 1E20, #default to 1E20
                                          valid_atoms = ['C', 'O', 'H', 'N', 'S', 'Si'], #allow silicon too
                                          fitting_iterations = 0, #loop through least-squares fitting until convergence, recommended
                                          save_name = 'name_of_source',
                                          overwrite_old_files = True) #if directory_path/save_name exists already, previous output files will be overwritten. Default is False.


assigner =results['assigner']
for _ in range(5):
    print()
print('============================================================')
print('Summary Statistics!')
print('============================================================')
print(results['statistics'])
print('vlsr', results['vlsr'])
print('temperature', results['temperature'])

for line in assigner.lines[:10]:  # First 10 lines
    if line.assignment_status:
        print(f"{line.frequency:.4f} MHz: {line.assignment_status.value}")
        if line.assigned_molecule:
            print(f"  → {line.assigned_molecule}")
            print(f"  Score: {line.best_candidate.global_score:.2f}")
            print('All Candidates:')
            candidate_list = [i.formula for i in line.candidates]
            print(candidate_list)
            print('')

In [ ]:
'''
Example in which single dish observation is used and neither the vlsr the temperature are known (so will be estimated by code).

Also: input a list of SMILES strings of known_molecules. These are the molecules which are used to initialize the weighted
chemical space surface for the structural relevance metric. Therefore, molecules similar to these inputted molecules will be initially more
highly scored from the structural relevance calculation.

In this example, the list includes formaldehyde, methanol, ethanol, methyl formate, 
dimethyl ether, acetone, and methyl cyanide, which is a reasonable starting set for hot core sources.
'''

results = astro_amase.assign_observations(spectrum_path='PATH TO SPECTRUM',
                                          directory_path = 'DIRECTORY PATH',
                                          temperature=170,  # Initial guess
                                          temperature_is_exact=False, #will estimate temperature automatically
                                          vlsr = None, #code will automatically determine vlsr
                                          sigma_threshold = 5, #5 sigma lines and stronger
                                          rms_noise = None, #will automatically determine noise level
                                          observation_type = 'single_dish',
                                          dish_diameter = 100, #meters
                                          continuum_temperature = 13,#default to 2.7
                                          source_size = 1E20, #default to 1E20
                                          valid_atoms = ['C', 'O', 'H', 'N', 'S'],
                                          fitting_iterations = 2,
                                          known_molecules = ['C=O','CO','CCO', 'COC=O', 'COC','CC(C)=O', 'CC#N']) #initialize structural relevance metric


assigner =results['assigner']
for _ in range(5):
    print()
print('============================================================')
print('Summary Statistics!')
print('============================================================')
print(results['statistics'])
print('vlsr', results['vlsr'])
print('temperature', results['temperature'])

for line in assigner.lines[:10]:  # First 10 lines
    if line.assignment_status:
        print(f"{line.frequency:.4f} MHz: {line.assignment_status.value}")
        if line.assigned_molecule:
            print(f"  → {line.assigned_molecule}")
            print(f"  Score: {line.best_candidate.global_score:.2f}")
            print('All Candidates:')
            candidate_list = [i.formula for i in line.candidates]
            print(candidate_list)
            print('')

In [ ]:
'''
Example in which single dish observation is used.
Temperature will be fixed at 150 K, but vlsr and rms noise will be determined automatically
'''

results = astro_amase.assign_observations(spectrum_path='PATH TO SPECTRUM',
                                          directory_path = 'DIRECTORY PATH',
                                          temperature=150,  #Input value
                                          temperature_is_exact=True, #temperature fixed
                                          vlsr = None, #code will automatically determine vlsr
                                          sigma_threshold = 5, #5 sigma lines and stronger
                                          rms_noise = None, #will automatically determine noise level
                                          observation_type = 'single_dish',
                                          dish_diameter = 100, #meters
                                          continuum_temperature = 2.7, #default to 2.7
                                          source_size = 1E20, #default to 1E20
                                          valid_atoms = ['C', 'O', 'H', 'N', 'S'])

assigner =results['assigner']
for _ in range(5):
    print()
print('============================================================')
print('Summary Statistics!')
print('============================================================')
print(results['statistics'])
print('vlsr', results['vlsr'])
print('temperature', results['temperature'])

for line in assigner.lines[:10]:  # First 10 lines
    if line.assignment_status:
        print(f"{line.frequency:.4f} MHz: {line.assignment_status.value}")
        if line.assigned_molecule:
            print(f"  → {line.assigned_molecule}")
            print(f"  Score: {line.best_candidate.global_score:.2f}")
            print('All Candidates:')
            candidate_list = [i.formula for i in line.candidates]
            print(candidate_list)
            print('')

## Force Include/Exclude Molecules

**Use cases:**
- **Force inclusion**: Useful when you know a molecule is present (e.g., from independent evidence) but the algorithm missed it or assigned it with low confidence
- **Force exclusion**: Helps when a molecule creates spurious assignments or you have strong evidence it's not present in your source

**Note**: The inputted molecule names must exactly match the molecule names in the `all_cdms_final_official.csv` and `all_jpl_final_official.csv` files.

In [ ]:
'''
Example in which single dish observation is used and none of the parameters are known.

The fitted spectrum will be forced to include CH3CN and forced to exclude HOCN.
The inputted molecule names for force_ignore_molecules and force_include_molecules must
match the molecule names in the all_cdms_final_official and all_jpl_final_official csv files.
'''

results = astro_amase.assign_observations(spectrum_path='PATH TO SPECTRUM',
                                          directory_path = 'DIRECTORY PATH',
                                          temperature=170,  # Initial guess
                                          temperature_is_exact=False, #will estimate temperature automatically
                                          vlsr = None, #code will automatically determine vlsr
                                          sigma_threshold = 5, #5 sigma lines and stronger
                                          rms_noise = None, #will automatically determine noise level
                                          observation_type = 'single_dish',
                                          dish_diameter = 100, #meters
                                          continuum_temperature = 2.7,#default to 2.7
                                          source_size = 1E20, #default to 1E20
                                          valid_atoms = ['C', 'O', 'H', 'N', 'S'],
                                          force_include_molecules = ['CH3CN'],
                                          force_ignore_molecules = ['HOCN'])

assigner =results['assigner']
for _ in range(5):
    print()
print('============================================================')
print('Summary Statistics!')
print('============================================================')
print(results['statistics'])
print('vlsr', results['vlsr'])
print('temperature', results['temperature'])

for line in assigner.lines[:10]:  # First 10 lines
    if line.assignment_status:
        print(f"{line.frequency:.4f} MHz: {line.assignment_status.value}")
        if line.assigned_molecule:
            print(f"  → {line.assigned_molecule}")
            print(f"  Score: {line.best_candidate.global_score:.2f}")
            print('All Candidates:')
            candidate_list = [i.formula for i in line.candidates]
            print(candidate_list)
            print('')

## Set VLSR Range and Molecules Used for VLSR and Temperature Determination

By default, the code searches for a vlsr value between -250 and 250 km/s. For many sources this range may be unnecessarily broad, and narrowing it can notably speed up the vlsr calculation. You can set a custom range using the `vlsr_range` parameter.

Similarly, all molecules listed in `/notebooks/vlsr_molecules.csv` are used by default when determining the vlsr and temperature. If you prefer to use only a subset of these species, you can specify them through the `vlsr_mols` parameter. The molecule names must match those provided in the .csv file. Reducing the list can also improve effiency.

The selected molecules are used for both the vlsr and temperature determination.


In [ ]:
'''
Example in which the vlsr_molecules and vlsr_range are defined 
'''

results = astro_amase.assign_observations(spectrum_path='PATH TO SPECTRUM',
                                          directory_path = 'DIRECTORY PATH',
                                          temperature=170,  # Initial guess
                                          temperature_is_exact=False, #will estimate temperature automatically
                                          vlsr = None, #code will automatically determine vlsr
                                          sigma_threshold = 5, #5 sigma lines and stronger
                                          rms_noise = None, #will automatically determine noise level
                                          observation_type = 'single_dish',
                                          dish_diameter = 100, #meters
                                          continuum_temperature = 2.7,#default to 2.7
                                          source_size = 1E20, #default to 1E20
                                          valid_atoms = ['C', 'O', 'H', 'N', 'S', 'Si'],
                                          vlsr_range = [-20,20], #vlsr range to consider as [min, max]
                                          vlsr_mols = ['H2CCO v = 0', 'OCS, v = 0', 'HC3N, (0,0,0,0)']) #molecules used to determine vlsr/temperature

# Inspecting Results

After running the analysis, you can load and inspect the output CSV files to examine the assignments and fitted parameters.

In [ ]:
# Load the output CSV files
peaks_df = pd.read_csv('DIRECTORY PATH/final_peak_results.csv')
column_densities = pd.read_csv('DIRECTORY PATH/column_density_results.csv')

# Display top assigned molecules by column density
print("Top 10 molecules by column density:")
print(column_densities.sort_values('column_density', ascending=False).head(10))

# Show some peak assignments
print("\nSample peak assignments:")
print(peaks_df.head(10))

# Filter for specific molecules
print("\nPeaks assigned to CH3OH:")
ch3oh_peaks = peaks_df[peaks_df['carrier_molecules'].str.contains('CH3OH', na=False)]
print(ch3oh_peaks[['peak_freq', 'experimental_intensity_max', 'carrier_molecules']])

The `molecule_summary()` function provides a per-molecule breakdown that reports how many lines each molecule is assigned to, along with whether it has at least one unblended line. It uses the `final_peak_results.csv` file produced during the assignment and can highlight molecules with weaker assignment confidence, such as those linked only to a single blended feature.

In [ ]:
molecule_dict = astro_amase.molecule_summary('DIRECTORY_PATH/final_peak_results.csv') #provide the final_peak_results.csv file

# Plotting Examples

The following cells demonstrate how to use the plotting functions to display/visualize the results of the assignment.

## Available Plotting Functions:

- **`show_fit_in_notebook(results)`**: Displays the interactive Bokeh plot inline in the notebook, showing the observed spectrum, total fitted spectrum, and individual molecular contributions. The plot allows toggling molecule visibility.

- **`plot_from_saved(...)`**: Recreates the interactive plot from previously saved results (useful for re-analyzing without re-running the full assignment).

- **`get_individual_plots(...)`**: Creates detailed PDF files for each assigned molecule, with subplots showing zoomed-in views of spectral peaks along with quantum number assignments.

- **`create_interactive_vlsr_plot(...)`**: Provides an interactive plot to visualize how changing the source velocity (VLSR) affects the simulated molecular spectrum. Can be useful to quickly test different VLSR values and confirm that the assigned VLSR is optimal.

In [ ]:
#First run the assignment code
results = astro_amase.assign_observations(spectrum_path='PATH TO SPECTRUM',
                                          directory_path = 'DIRECTORY PATH',
                                          temperature=150,  # Initial guess
                                          temperature_is_exact=False, #will estimate temperature automatically
                                          vlsr = None, #code will automatically determine vlsr
                                          sigma_threshold = 5, #5 sigma lines and stronger
                                          rms_noise = None, #will automatically determine noise level
                                          observation_type = 'interferometric',
                                          beam_major_axis = 0.4, #arcsec
                                          beam_minor_axis = 0.4, #arcsec
                                          continuum_temperature = 13,#default to 2.7
                                          source_size = 1E20, #default to 1E20
                                          valid_atoms = ['C', 'O', 'H', 'N', 'S'])

#this will load the interactive result plot in the notebook (including all assigned molecules)
#note that this is also saved as a .html file to the directory_path automatically when the code is run
astro_amase.show_fit_in_notebook(results)

#to visualize a specific set of assigned molecules:
astro_amase.show_fit_in_notebook(results, mols_to_display=['H2CO', 'H2S'])

In [ ]:
'''
This code will allow you to plot the interactive plot from a previous run.
You will need to provide the spectrum path as well as the paths to the following files
that are created during the assignment:

- column_density_results.csv
- analysis_parameters.json 
'''

astro_amase.plot_from_saved(
    spectrum_path='spectrum.txt',
    directory_path='DIRECTORY PATH/',
    column_density_csv='DIRECTORY PATH/column_density_results.csv',
    stored_json='DIRECTORY PATH/analysis_parameters.json'
)

#you can also select specific molecules to display

astro_amase.plot_from_saved(
    spectrum_path='spectrum.txt',
    directory_path='DIRECTORY PATH/',
    column_density_csv='DIRECTORY PATH/column_density_results.csv',
    stored_json='DIRECTORY PATH/analysis_parameters.json',
    mols_to_display=['CH3OH, vt = 0 - 2']
)

 

In [ ]:
'''
This function creates PDF files containing subplots of spectral peaks for each assigned
molecule. Each subplot shows a zoomed-in view around a detected peak, displaying both
the observed spectrum and the simulated spectrum for comparison, along with quantum
number assignments. The PDF files are saved to the directory_path (which must contain the saved files from Dropbox).

You will need to provide the spectrum path as well as the paths to the following files
that are created during the assignment:

- column_density_results.csv
- analysis_parameters.json 
'''

astro_amase.get_individual_plots(
    spectrum_path='spectrum.txt',
    directory_path='DIRECTORY PATH',
    column_density_csv='DIRECTORY PATH/column_density_results.csv',
    stored_json='DIRECTORY PATH/analysis_parameters.json',
    minimum_intensity='default'  # or specify a custom threshold
)

#can specify molecules to create plots for:
astro_amase.get_individual_plots(
    spectrum_path='spectrum.txt',
    directory_path='DIRECTORY PATH',
    column_density_csv='DIRECTORY PATH/column_density_results.csv',
    stored_json='DIRECTORY PATH/analysis_parameters.json',
    minimum_intensity='default',
    mols_to_display=['c-C2H3O']
)


In [ ]:
'''
This function creates an interactive Bokeh plot that allows you to test different VLSR 
(source velocity) values using a slider and see how they affect the alignment between 
observed and simulated molecular spectra. The plot displays the observed spectrum in 
black and the simulated spectrum in red.

The function computes the molecular spectrum once at VLSR=0 km/s, then applies fast 
Doppler shifts as you drag the slider. 

To use the interactive plot:
1. Drag the slider left/right to adjust VLSR
2. The simulated spectrum shifts in real-time
3. Use the built-in Bokeh tools to pan, zoom, and examine features in detail
4. Click legend items to hide/show spectra

You will need to provide the spectrum path and directory_path (which must contain the 
saved Dropbox catalog files). You also need to specify the molecule name (which must match the saved .csv files), excitation 
temperature, linewidth, and other source parameters.

Optional slider parameters:
- vlsr_min, vlsr_max: Set the range of the slider (default: -50 to 50 km/s)
- vlsr_step: Set the slider resolution (default: 0.05 km/s
'''


# Example 1: Basic usage with defaults
astro_amase.create_interactive_vlsr_plot(
    spectrum_path='PATH TO SPECTRUM',
    directory_path='DIRECTORY PATH',
    molecule_name='CH3OH, vt = 0 - 2',  # Methanol
    excitation_temperature=20.0,
    linewidth=2.5,
    column_density=1.0e15,
    observation_type='single_dish',
    dish_diameter=100
)

# Example 2: More parameters defined
astro_amase.create_interactive_vlsr_plot(
    spectrum_path='PATH TO SPECTRUM',
    directory_path='DIRECTORY PATH',
    molecule_name='HC3N, (0,0,0,0)',  # Methyl cyanide
    excitation_temperature=150.0,
    linewidth=5.0,
    column_density=1.0e16,
    vlsr_min=0,      # Range from 0 to 50 km/s
    vlsr_max=50,
    vlsr_step=0.01,   # Fine control (0.01 km/s steps)
    observation_type='interferometric',
    beam_major_axis=0.3,
    beam_minor_axis=0.3,
    source_size = 1.E20
)


# Molecule Prediction Examples

The molecule prediction feature uses a neural network-based approach to suggest new candidate molecules that are likely to exist in your source based on the molecules you've already detected. This is useful for:
- Guiding follow-up observations
- Identifying chemically related molecules to search for
- Exploring the chemical complexity of your source

## How It Works:
1. **Fragment-based prediction**: Uses a trained neural network to predict molecular fragments
2. **Chemical space sampling**: Samples latent space around your detected molecules
3. **Stochastic assembly**: Builds candidate molecules from predicted fragments
4. **Gaussian scoring**: Ranks candidates by their proximity to detected molecules in chemical space

The algorithm matches the chemical properties (radicals, ions, heteroatom placement) of your detected set and returns candidates ranked by likelihood.

In [ ]:
'''
Basic molecule prediction example.
This will predict new molecules based on the detected molecules from your analysis.
'''

# First, get the detected molecules from your results
column_densities = pd.read_csv('DIRECTORY PATH/column_density_results.csv')
detected_smiles = column_densities['smiles'].tolist()
unique_smiles = list(set(detected_smiles)) #remove duplicates

# Run molecule prediction
predicted_molecules = astro_amase.molecule_prediction(
    directory_path='DIRECTORY PATH',  # Same directory with Dropbox files
    detected_smiles=unique_smiles,
    valid_atoms='default'  # Automatically infer valid atoms from detected molecules
)

# Display top 100 predicted molecules
print("Top 100 Predicted Molecules (ranked by likelihood):")
print("="*60)
print(predicted_molecules[:100])

In [ ]:
'''
Molecule prediction with custom detected molecules.
You can also provide your own list of known molecules (as SMILES strings)
rather than using the automatic detection results.
'''


# Example: Predict molecules based on a known set of hot core COMs
my_detected_molecules = [
    'C=O',          # formaldehyde
    'CO',           # methanol  
    'CCO',          # ethanol
    'COC',          # dimethyl ether
    'CC(C)=O',      # acetone
    'CC#N',         # methyl cyanide
    'COC=O',     # methyl formate
]

# Run prediction with restricted atom types
predicted_molecules = astro_amase.molecule_prediction(
    directory_path='DIRECTORY PATH',
    detected_smiles=my_detected_molecules,
    valid_atoms=['C', 'H', 'O', 'N']  #Discard molecules with atoms other than C,H,O,N
)

# Display top 100 predicted molecules
print("Top 100 Predicted Molecules (ranked by likelihood):")
print("="*60)
print(predicted_molecules[:100])

# Parameter Determination Examples

The following code finds the source parameters (i.e. excitation temperature, vlsr, dV) but does not assign the lines.

In [ ]:
'''
Just finding the linewidth
'''
results = astro_amase.get_linewidth(spectrum_path='PATH TO SPECTRUM')

print('median velocity linewidth:', results['linewidth_kms'])
print('median frequency linewidth:', results['linewidth_mhz'])

In [ ]:
'''
Finding all source parameters (linewidth, vlsr, temperature), not assigning lines.
'''

astro_amase.get_source_parameters(spectrum_path='PATH TO SPECTRUM',
                                   directory_path = 'DIRECTORY PATH',
                                   temperature= 150,  # Initial guess, can vary up to 100K away from this value
                                   observation_type = 'single_dish',
                                   dish_diameter = 100,
                                   continuum_temperature = 2.7)